In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA, NMF
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
# from scipy.stats import ttest_ind, chi2_contingency, pearsonr
# from statsmodels.stats.multitest import multipletests
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Input, Dense, concatenate, Lambda
from tensorflow.keras.models import Model
from keras import backend as K


2024-08-15 19:52:23.450920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 19:52:23.467625: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 19:52:23.472746: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 19:52:23.484986: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-15 19:52:24.202085: W tensorflow/compiler/tf2

In [2]:
def find_binary_columns(X_train):
    binary_columns = []
    for col in range(X_train.shape[1]):
        unique_values = np.unique(X_train[:, col])
        if set(unique_values).issubset({0, 1}):
            binary_columns.append(col)
    return binary_columns

In [3]:
dataset_path = "dataset_amyloid.pickle"
with open(dataset_path, 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)
X = np.concatenate((X_train, X_test), axis=0)
Y = np.concatenate((y_train, y_test), axis=0)

binary_columns = find_binary_columns(X)

numerical_columns = []
for i in range(0,10193):
    if i not in binary_columns:
        numerical_columns.append(i)

X_bi = X[:, binary_columns] # Binary dataset
X_nu = X[:, numerical_columns] # Numerical dataset

In [4]:
print(f'Number of Binary columns = {X_bi.shape}')
print(f'Number of Numerical columns = {X_nu.shape}')
print(f'Minimum value of numerical element = {np.min(X_nu)}')
print(f'Maximum value of numerical element = {np.max(X_nu)}')

Number of Binary columns = (190, 3887)
Number of Numerical columns = (190, 6306)
Minimum value of numerical element = 0.0
Maximum value of numerical element = 5347441.0


In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_nu)

print(f'Minimum value of scaled numerical element = {np.min(X_scaled)}')
print(f'Maximum value of scaled numerical element = {np.max(X_scaled)}')

# label_encoder = LabelEncoder()
# Y_numerical = label_encoder.fit_transform(Y)

Minimum value of scaled numerical element = -1.22930590661059
Maximum value of scaled numerical element = 13.74772708486753


In [6]:
def pca_model(X, n_component):
    print(f'Method: PCA')
    pca = PCA(n_components=n_component)
    X_reduced = pca.fit_transform(X)
    X_reconstructed = pca.inverse_transform(X_reduced)
    
    print(f'Number of components selected: {pca.n_components_}')
    mse = mean_squared_error(X.flatten(), X_reconstructed.flatten())
    print(f'Mean Squared Error (MSE): {mse}')
    
    return X_reduced

def nmf_model(X, n_component):
    print(f'Method: NMF')
    nmf = NMF(n_components=n_component, init='random', random_state=0)
    X_reduced = nmf.fit_transform(X)
    X_reconstructed = nmf.inverse_transform(X_reduced)
    
    print(f'Number of components selected: {nmf.n_components_}')
    mse = mean_squared_error(X.flatten(), X_reconstructed.flatten())
    print(f'Mean Squared Error (MSE): {mse}')

    return X_reduced

def svm_model(X, Y, kernel="linear"):
    print(f'Method: SVM')
    svm = SVC(kernel=kernel)
    svm.fit(X, Y)
    
    selector = SelectFromModel(svm, prefit=True)
    X_reduced = selector.transform(X)

    print("Number of coponents selected:", X_reduced.shape[1])

    return X_reduced

def rf_model(X, Y, threshold, n_estimators=100, random_state=42):
    print(f'Method: Random Forest')
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    rf.fit(X, Y)
    
    selector = SelectFromModel(rf, threshold=threshold, prefit=True)
    X_reduced = selector.transform(X)
    
    selected_features_indices = selector.get_support(indices=True)

    print(f'Random Forest Threshold: {threshold}')
    print(f'Number of coponents selected: {X_reduced.shape[1]}')
    # print(f'Selected Features: {selected_features_indices}')
    
    return X_reduced #, selected_features_indices

In [7]:
print(f'----Numerical Dataset----')
pca_data = pca_model(X_scaled, 128)

print(f'----Binary Dataset----')
nmf_data = nmf_model(X_bi, 64)
svm_data = svm_model(X_bi, Y)
rf_threshold = 0.003
rf_data = rf_model(X_bi, Y, rf_threshold)

# print(pca_data.shape)

----Numerical Dataset----
Method: PCA
Number of components selected: 128
Mean Squared Error (MSE): 0.03259457373367942
----Binary Dataset----
Method: NMF
Number of components selected: 64
Mean Squared Error (MSE): 0.0011488281093814806
Method: SVM
Number of coponents selected: 916
Method: Random Forest
Random Forest Threshold: 0.003
Number of coponents selected: 65


In [8]:
# Numerical Dataset: 
# Method: PCA
# Number of components selected: 128
# Mean Squared Error (MSE): 0.032486597153881774
# Binary Dataset: 
# Method: NMF
# Number of components selected: 64
# Mean Squared Error (MSE): 0.0011488281093814806
# Method: SVM
# Number of coponents selected: 916
# Method: Random Forest
# Random Forest Threshold: mean
# Number of coponents selected: 754

In [9]:
# def late_fusion_features(X_1, X_2):

#     # Define the input shapes based on the data
#     input_shape_1 = X_1.shape[1]
#     input_shape_2 = X_2.shape[1]

#     # Define the first input stream for numerical data
#     input_1 = Input(shape=(input_shape_1,), name='input_1')
#     dense_1 = Dense(64, activation='relu')(input_1)
#     dense_1 = Dense(32, activation='relu')(dense_1)
#     output_1 = Dense(16, activation='relu', name='output_1')(dense_1)

#     # Define the second input stream for binary data
#     input_2 = Input(shape=(input_shape_2,), name='input_2')
#     dense_2 = Dense(64, activation='relu')(input_2)
#     dense_2 = Dense(32, activation='relu')(dense_2)
#     output_2 = Dense(16, activation='relu', name='output_2')(dense_2)

#     # Late fusion via concatenation of features
#     fused_features = concatenate([output_1, output_2], name='fused_features')

#     # Create the model
#     model = Model(inputs=[input_1, input_2], outputs=fused_features)

#     # Compile the model (necessary to run Keras models)
#     model.compile(optimizer='adam', loss='mse')  # Using mse as a placeholder loss

#     # Use the model to predict the fused features
#     X_fusion = model.predict([X_1, X_2])

#     # Clean up Keras session
#     K.clear_session()

#     return X_fusion


In [10]:
 def select_confident_features(x):
        k = 8  # number of top features to select from each stream
        x_sorted = tf.sort(tf.abs(x), direction='DESCENDING')
        x_top_k = x_sorted[:, :k]
        return x_top_k
     
def late_fusion_features(X_1, X_2):

    device = '/gpu:0' if tf.config.list_physical_devices('GPU') else '/cpu:0'
    print(device)
    with tf.device(device):
        # Define the input shapes based on the data
        input_shape_1 = X_1.shape[1]
        input_shape_2 = X_2.shape[1]

        # Define the first input stream for the first feature set
        input_1 = Input(shape=(input_shape_1,), name='input_1')
        dense_1 = Dense(64, activation='relu')(input_1)
        dense_1 = Dense(32, activation='relu')(dense_1)
        output_1 = Dense(16, activation='relu', name='output_1')(dense_1)

        # Define the second input stream for the second feature set
        input_2 = Input(shape=(input_shape_2,), name='input_2')
        dense_2 = Dense(64, activation='relu')(input_2)
        dense_2 = Dense(32, activation='relu')(dense_2)
        output_2 = Dense(16, activation='relu', name='output_2')(dense_2)

        top_features_1 = Lambda(select_confident_features)(output_1)
        top_features_2 = Lambda(select_confident_features)(output_2)

        # Late fusion via concatenation of the most confident features
        fused_features = concatenate([top_features_1, top_features_2], name='fused_features')
    
        # Create the model
        model = Model(inputs=[input_1, input_2], outputs=fused_features)
    
        # Use the model to predict the fused features
        X_fusion = model.predict([X_1, X_2])

    return X_fusion

In [ ]:
X_fusion = late_fusion_features(X_nu, X_bi)

/gpu:0


2024-08-15 19:52:45.782273: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79078 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:0b:00.0, compute capability: 8.0
W0000 00:00:1723751566.662418    1279 gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
W0000 00:00:1723751566.663527    1274 gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
W0000 00:00:1723751566.664612    1276 gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
W0000 00:00:1723751566.665700    1275 gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
W0000 00:00:1723751566.666819    1282 gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back 

In [ ]:
print(Y.shape)

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, Concatenate
# from tensorflow.keras.models import Model

# # Numerical data input and processing
# input_pca = Input(shape=(128,), name='pca_input')
# x_pca = Dense(64, activation='relu', name='pca_dense1')(input_pca)
# x_pca = Dense(32, activation='relu', name='pca_dense2')(x_pca)

# # Binary data input and processing
# input_nmf = Input(shape=(64,), name='nmf_input')
# x_nmf = Dense(32, activation='relu', name='nmf_dense1')(input_nmf)
# x_nmf = Dense(16, activation='relu', name='nmf_dense2')(x_nmf)

# # Feature fusion
# combined = Concatenate(name='concatenate')([x_pca, x_nmf])
# output = Dense(1, activation='sigmoid', name='output')(combined)

# # Model definition
# model = Model(inputs=[input_pca, input_nmf], outputs=output)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

# # Example data
# # import numpy as np
# # pca_data = np.random.rand(190, 124)  # Replace with actual pca_data
# # nmf_data = np.random.randint(2, size=(190, 64))  # Replace with actual nmf_data
# # Y = np.random.randint(2, size=190)  # Replace with actual target data

# # Model training
# model.fit([pca_data, nmf_data], Y, epochs=10, batch_size=32)


In [ ]:
# ———— New New Version
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class MultimodalModel(nn.Module):
    def __init__(self):
        super(MultimodalModel, self).__init__()
        # Numerical data processing layers
        self.pca_dense1 = nn.Linear(128, 64)
        self.pca_dense2 = nn.Linear(64, 32)
        # Binary data processing layers
        self.nmf_dense1 = nn.Linear(64, 32)
        self.nmf_dense2 = nn.Linear(32, 16)
        # Combined layers
        self.combined_dense1 = nn.Linear(32 + 16, 16)
        self.combined_dense2 = nn.Linear(16, 1)

    def forward(self, pca_input, nmf_input):
        # Process numerical data
        x_pca = torch.relu(self.pca_dense1(pca_input))
        x_pca = torch.relu(self.pca_dense2(x_pca))
        # Process binary data
        x_nmf = torch.relu(self.nmf_dense1(nmf_input))
        x_nmf = torch.relu(self.nmf_dense2(x_nmf))
        # Concatenate and process combined features
        combined = torch.cat((x_pca, x_nmf), dim=1)
        x_combined = torch.relu(self.combined_dense1(combined))
        output = torch.sigmoid(self.combined_dense2(x_combined))
        return output, combined

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

model = MultimodalModel().to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

pca_torch = torch.tensor(pca_data, dtype=torch.float32).to(device)
nmf_torch = torch.tensor(nmf_data, dtype=torch.float32).to(device)
Y_torch = torch.tensor(Y, dtype=torch.float32).to(device)

num_epochs = 10
batch_size = 32
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for i in range(0, len(Y), batch_size):
        pca_batch = pca_torch[i:i + batch_size]
        nmf_batch = nmf_torch[i:i + batch_size]
        y_batch = Y_torch[i:i + batch_size]

        # Forward pass
        outputs, combined_features = model(pca_batch, nmf_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        total_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(Y)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')

model.eval()
with torch.no_grad():
    final_output, combined_features = model(pca_torch, nmf_torch)
    # print(f'Final Output: {final_output}')
    # print(f'Combined Features: {combined_features}')
print(combined_features.shape)

In [ ]:
print(combined_features)

In [ ]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from fastFM import sgd

# Combine data into a single feature set
combined_data = np.hstack((pca_data, nmf_data))

# Convert to dictionary format for DictVectorizer
data_dict = [dict(enumerate(row)) for row in combined_data]

vec = DictVectorizer()
X = vec.fit_transform(data_dict)

Y = np.random.randint(2, size=190)  # Replace with actual target data

# Train Factorization Machine
fm = sgd.FMClassification(n_iter=100, init_stdev=0.1, l2_reg_w=0.1, l2_reg_V=0.1, rank=10)
fm.fit(X, Y)


In [5]:
# ----------------------------------------------------------------------------
# Copyright (c) 2019--, gemelli development team.
#
# Distributed under the terms of the Modified BSD License.
#
# The full license is in the file COPYING.txt, distributed with this software.
# ----------------------------------------------------------------------------

import biom
import skbio
import warnings
import numpy as np
import pandas as pd
from scipy.spatial import distance
from typing import Union, Optional
from skbio import TreeNode, OrdinationResults, DistanceMatrix
from gemelli.matrix_completion import MatrixCompletion
from gemelli.optspace import OptSpace
from gemelli.preprocessing import (matrix_rclr,
                                   fast_unifrac,
                                   bp_read_phylogeny,
                                   retrieve_t2t_taxonomy,
                                   create_taxonomy_metadata,
                                   mask_value_only)
from gemelli._defaults import (DEFAULT_COMP, DEFAULT_MTD,
                               DEFAULT_MSC, DEFAULT_MFC,
                               DEFAULT_OPTSPACE_ITERATIONS,
                               DEFAULT_MFF, DEFAULT_METACV,
                               DEFAULT_COLCV, DEFAULT_TESTS,
                               DEFAULT_MATCH, DEFAULT_TRNSFRM)
from scipy.linalg import svd
# import QIIME2 if in a Q2env otherwise set type to str
try:
    from q2_types.tree import NewickFormat
except ImportError:
    # python does not check but technically this is the type
    NewickFormat = str


def phylogenetic_rpca_without_taxonomy(
        table: biom.Table,
        phylogeny: NewickFormat,
        n_components: Union[int, str] = DEFAULT_COMP,
        min_sample_count: int = DEFAULT_MSC,
        min_feature_count: int = DEFAULT_MFC,
        min_feature_frequency: float = DEFAULT_MFF,
        min_depth: int = DEFAULT_MTD,
        max_iterations: int = DEFAULT_OPTSPACE_ITERATIONS) -> (
        OrdinationResults, DistanceMatrix,
        TreeNode, biom.Table):
    """
    Runs phylogenetic RPCA without taxonomy. This code will
    be run QIIME2 versions of gemelli. Outside of QIIME2
    please use phylogenetic_rpca.
    """

    output = phylogenetic_rpca(table=table,
                               phylogeny=phylogeny,
                               n_components=n_components,
                               min_sample_count=min_sample_count,
                               min_feature_count=min_feature_count,
                               min_feature_frequency=min_feature_frequency,
                               min_depth=min_depth,
                               max_iterations=max_iterations)
    ord_res, dist_res, phylogeny, counts_by_node, _ = output

    return ord_res, dist_res, phylogeny, counts_by_node


def phylogenetic_rpca_with_taxonomy(
            table: biom.Table,
            phylogeny: NewickFormat,
            taxonomy: pd.DataFrame,
            n_components: Union[int, str] = DEFAULT_COMP,
            min_sample_count: int = DEFAULT_MSC,
            min_feature_count: int = DEFAULT_MFC,
            min_feature_frequency: float = DEFAULT_MFF,
            min_depth: int = DEFAULT_MTD,
            max_iterations: int = DEFAULT_OPTSPACE_ITERATIONS) -> (
        OrdinationResults, DistanceMatrix,
        TreeNode, biom.Table, pd.DataFrame):
    """
    Runs phylogenetic RPCA with taxonomy. This code will
    be run QIIME2 versions of gemelli. Outside of QIIME2
    please use phylogenetic_rpca.
    """

    output = phylogenetic_rpca(table=table,
                               phylogeny=phylogeny,
                               taxonomy=taxonomy,
                               n_components=n_components,
                               min_sample_count=min_sample_count,
                               min_feature_count=min_feature_count,
                               min_feature_frequency=min_feature_frequency,
                               min_depth=min_depth,
                               max_iterations=max_iterations)
    ord_res, dist_res, phylogeny, counts_by_node, result_taxonomy = output

    return ord_res, dist_res, phylogeny, counts_by_node, result_taxonomy


def phylogenetic_rpca(table: biom.Table,
                      phylogeny: NewickFormat,
                      taxonomy: Optional[pd.DataFrame] = None,
                      n_components: Union[int, str] = DEFAULT_COMP,
                      min_sample_count: int = DEFAULT_MSC,
                      min_feature_count: int = DEFAULT_MFC,
                      min_feature_frequency: float = DEFAULT_MFF,
                      min_depth: int = DEFAULT_MTD,
                      max_iterations: int = DEFAULT_OPTSPACE_ITERATIONS) -> (
                          OrdinationResults, DistanceMatrix,
                          TreeNode, biom.Table, Optional[pd.DataFrame]):
    """
    Performs robust phylogenetic center log-ratio transform and
    robust PCA. The robust PCA and enter log-ratio transform
    operate on only observed values of the data.
    For more information see (1 and 2).

    Parameters
    ----------
    table: numpy.ndarray, required
    The feature table in biom format containing the
    samples over which metric should be computed.

    phylogeny: str, required
    Path to the file containing the phylogenetic tree containing tip
    identifiers that correspond to the feature identifiers in the table.
    This tree can contain tip ids that are not present in the table,
    but all feature ids in the table must be present in this tree.

    taxonomy: pd.DataFrame, optional
    Taxonomy file in QIIME2 formatting. See the feature metdata
    section of https://docs.qiime2.org/2021.11/tutorials/metadata

    n_components: int, optional : Default is 3
    The underlying rank of the data and number of
    output dimensions.

    min_sample_count: int, optional : Default is 0
    Minimum sum cutoff of sample across all features.
    The value can be at minimum zero and must be an
    whole integer. It is suggested to be greater than
    or equal to 500.

    min_feature_count: int, optional : Default is 0
    Minimum sum cutoff of features across all samples.
    The value can be at minimum zero and must be
    an whole integer.

    min_feature_frequency: float, optional : Default is 0
    Minimum percentage of samples a feature must appear
    with a value greater than zero. This value can range
    from 0 to 100 with decimal values allowed.

    max_iterations: int, optional : Default is 5
    The number of convex iterations to optimize the solution
    If iteration is not specified, then the default iteration is 5.
    Which reduces to a satisfactory error threshold.

    Returns
    -------
    OrdinationResults
        A biplot of the (Robust Aitchison) RPCA feature loadings

    DistanceMatrix
        The Aitchison distance of the sample loadings from RPCA.

    TreeNode
        The input tree with all nodes matched in name to the
        features in the counts-by-node table

    biom.Table
        A table with all tree internal nodes as features with the
        sum of all children of that node (i.e. FastUniFrac).

    Optional[pd.DataFrame]
        The resulting tax2Tree taxonomy and will include taxonomy for both
        internal nodes and tips. Note: this will only be output
        if taxonomy was given as input.

    Raises
    ------
    ValueError
        `ValueError: n_components must be at least 2`.

    ValueError
        `ValueError: max_iterations must be at least 1`.

    ValueError
        `ValueError: Data-table contains either np.inf or -np.inf`.

    ValueError
        `ValueError: The n_components must be less
            than the minimum shape of the input table`.

    References
    ----------
    .. [1] Martino C, Morton JT, Marotz CA, Thompson LR, Tripathi A,
           Knight R, Zengler K. 2019. A Novel Sparse Compositional
           Technique Reveals Microbial Perturbations. mSystems 4.
    .. [2] Keshavan RH, Oh S, Montanari A. 2009. Matrix completion
            from a few entries (2009_ IEEE International
            Symposium on Information Theory

    Examples
    --------
    import numpy as np
    import pandas as pd
    from biom import Table
    from gemelli.rpca import phylogenetic_rpca

    # make a table
    X = np.array([[9, 3, 0, 0],
                [9, 9, 0, 1],
                [0, 1, 4, 5],
                [0, 0, 3, 4],
                [1, 0, 8, 9]])
    sample_ids = ['s1','s2','s3','s4']
    feature_ids = ['f1','f2','f3','f4','f5']
    bt = Table(X, feature_ids, sample_ids)
    # write an example tree to read
    f = open("demo-tree.nwk", "w")
    newick = '(((f1:1,f2:1)n9:1,f3:1)n8:1,(f4:1,f5:1)n2:1)n1:1;'
    f.write(newick)
    f.close()
    # run RPCA without taxonomy
    # s1/s2 will seperate from s3/s4
    (ordination, distance_matrix,
    tree, phylo_table, _) = phylogenetic_rpca(bt, 'demo-tree.nwk')

    # make mock taxonomy
    taxonomy = pd.DataFrame({fid:['k__kingdom; p__phylum;'
                                'c__class; o__order; '
                                'f__family; g__genus;'
                                's__',
                                0.99]
                            for fid in feature_ids},
                            ['Taxon', 'Confidence']).T
    # run RPCA with taxonomy
    # s1/s2 will seperate from s3/s4
    (ordination, distance_matrix,
    tree, phylo_table,
    lca_taxonomy) = phylogenetic_rpca(bt, 'demo-tree.nwk', taxonomy)

    """

    # validate the metadata using q2 as a wrapper
    if taxonomy is not None and not isinstance(taxonomy,
                                               pd.DataFrame):
        taxonomy = taxonomy.to_dataframe()
    # use helper to process table
    table = rpca_table_processing(table,
                                  min_sample_count,
                                  min_feature_count,
                                  min_feature_frequency)

    # import the tree based on filtered table
    phylogeny = bp_read_phylogeny(table,
                                  phylogeny,
                                  min_depth)
    # build the vectorized table
    counts_by_node, tree_index, branch_lengths, fids, otu_ids\
        = fast_unifrac(table, phylogeny)
    # Robust-clt (matrix_rclr) preprocessing
    rclr_table = matrix_rclr(counts_by_node, branch_lengths=branch_lengths)
    # run OptSpace (RPCA)
    ord_res, dist_res = optspace_helper(rclr_table, fids, table.ids(),
                                        n_components=n_components)
    # import expanded table
    counts_by_node = biom.Table(counts_by_node.T,
                                fids, table.ids())
    result_taxonomy = None
    if taxonomy is not None:
        # collect taxonomic information for all tree nodes.
        traversed_taxonomy = retrieve_t2t_taxonomy(phylogeny, taxonomy)
        result_taxonomy = create_taxonomy_metadata(phylogeny,
                                                   traversed_taxonomy)

    return ord_res, dist_res, phylogeny, counts_by_node, result_taxonomy


def rpca(table: biom.Table,
         n_components: Union[int, str] = DEFAULT_COMP,
         min_sample_count: int = DEFAULT_MSC,
         min_feature_count: int = DEFAULT_MFC,
         min_feature_frequency: float = DEFAULT_MFF,
         max_iterations: int = DEFAULT_OPTSPACE_ITERATIONS) -> (
        OrdinationResults,
        DistanceMatrix):
    """
    Performs robust center log-ratio transform and
    robust PCA. The robust PCA and enter log-ratio transform
    operate on only observed values of the data.
    For more information see (1 and 2).

    Parameters
    ----------
    table: numpy.ndarray, required
    The feature table in biom format containing the
    samples over which metric should be computed.

    n_components: int, optional : Default is 3
    The underlying rank of the data and number of
    output dimensions.

    min_sample_count: int, optional : Default is 0
    Minimum sum cutoff of sample across all features.
    The value can be at minimum zero and must be an
    whole integer. It is suggested to be greater than
    or equal to 500.

    min_feature_count: int, optional : Default is 0
    Minimum sum cutoff of features across all samples.
    The value can be at minimum zero and must be
    an whole integer.

    min_feature_frequency: float, optional : Default is 0
    Minimum percentage of samples a feature must appear
    with a value greater than zero. This value can range
    from 0 to 100 with decimal values allowed.

    max_iterations: int, optional : Default is 5
    The number of convex iterations to optimize the solution
    If iteration is not specified, then the default iteration is 5.
    Which reduces to a satisfactory error threshold.

    Returns
    -------
    OrdinationResults
        A biplot of the (Robust Aitchison) RPCA feature loadings

    DistanceMatrix
        The Aitchison distance of the sample loadings from RPCA.

    Raises
    ------
    ValueError
        `ValueError: n_components must be at least 2`.

    ValueError
        `ValueError: max_iterations must be at least 1`.

    ValueError
        `ValueError: Data-table contains either np.inf or -np.inf`.

    ValueError
        `ValueError: The n_components must be less
            than the minimum shape of the input table`.

    References
    ----------
    .. [1] Martino C, Morton JT, Marotz CA, Thompson LR, Tripathi A,
           Knight R, Zengler K. 2019. A Novel Sparse Compositional
           Technique Reveals Microbial Perturbations. mSystems 4.
    .. [2] Keshavan RH, Oh S, Montanari A. 2009. Matrix completion
            from a few entries (2009_ IEEE International
            Symposium on Information Theory

    Examples
    --------
    import numpy as np
    from biom import Table
    from gemelli.rpca import rpca

    # make a table
    X = np.array([[9, 3, 0, 0],
                [9, 9, 0, 1],
                [0, 1, 4, 5],
                [0, 0, 3, 4],
                [1, 0, 8, 9]])
    sample_ids = ['s1','s2','s3','s4']
    feature_ids = ['f1','f2','f3','f4','f5']
    bt = Table(X, feature_ids, sample_ids)
    # run RPCA (s1/s2 will seperate from s3/s4)
    ordination, distance_matrix = rpca(bt)

    """
    # use helper to process table
    table = rpca_table_processing(table,
                                  min_sample_count,
                                  min_feature_count,
                                  min_feature_frequency)
    # Robust-clt (matrix_rclr) preprocessing
    rclr_table = matrix_rclr(table.matrix_data.toarray().T)
    # run OptSpace (RPCA)
    ord_res, dist_res = optspace_helper(rclr_table,
                                        table.ids('observation'),
                                        table.ids(), n_components=n_components)

    return ord_res, dist_res


def rpca_with_cv(table: biom.Table,
                 n_test_samples: int = DEFAULT_TESTS,
                 sample_metadata: pd.DataFrame = DEFAULT_METACV,
                 train_test_column: str = DEFAULT_COLCV,
                 n_components: Union[int, str] = DEFAULT_COMP,
                 max_iterations: int = DEFAULT_OPTSPACE_ITERATIONS,
                 min_sample_count: int = DEFAULT_MSC,
                 min_feature_count: int = DEFAULT_MFC,
                 min_feature_frequency: float = DEFAULT_MFF) -> (
                 OrdinationResults,
                 DistanceMatrix,
                 pd.DataFrame):
    """
    RPCA but with CV used in Joint-RPCA.

    Parameters
    ----------
    table: numpy.ndarray, required
    The feature table in biom format containing the
    samples over which metric should be computed.

    n_test_samples: int, optional : Default is 10
    Number of random samples to choose for test split samples.

    metadata: DataFrame, optional : Default is None
    Sample metadata file in QIIME2 formatting. The file must
    contain a train-test column with labels `train` and `test`
    and the row ids matched to the table(s).

    train_test_column: str, optional : Default is None
    Sample metadata column containing `train` and `test`
    labels to use for the cross-validation evaluation.

    n_components: int, optional : Default is 3
    The underlying rank of the data and number of
    output dimensions.

    min_sample_count: int, optional : Default is 0
    Minimum sum cutoff of sample across all features.
    The value can be at minimum zero and must be an
    whole integer. It is suggested to be greater than
    or equal to 500.

    min_feature_count: int, optional : Default is 0
    Minimum sum cutoff of features across all samples.
    The value can be at minimum zero and must be
    an whole integer.

    min_feature_frequency: float, optional : Default is 0
    Minimum percentage of samples a feature must appear
    with a value greater than zero. This value can range
    from 0 to 100 with decimal values allowed.

    max_iterations: int, optional : Default is 5
    The number of convex iterations to optimize the solution
    If iteration is not specified, then the default iteration is 5.
    Which reduces to a satisfactory error threshold.

    Returns
    -------
    OrdinationResults
        A biplot of the (Robust Aitchison) RPCA feature loadings

    DistanceMatrix
        The Aitchison distance of the sample loadings from RPCA.

    DataFrame
        The cross-validation reconstruction error.

    Raises
    ------
    ValueError
        `ValueError: n_components must be at least 2`.

    ValueError
        `ValueError: max_iterations must be at least 1`.

    ValueError
        `ValueError: Data-table contains either np.inf or -np.inf`.

    ValueError
        `ValueError: The n_components must be less
            than the minimum shape of the input table`.

    """
    res_tmp = joint_rpca([table],
                         n_test_samples=n_test_samples,
                         sample_metadata=sample_metadata,
                         train_test_column=train_test_column,
                         n_components=n_components,
                         max_iterations=max_iterations,
                         min_sample_count=min_sample_count,
                         min_feature_count=min_feature_count,
                         min_feature_frequency=min_feature_frequency)
    ord_res, dist_res, cv_dist = res_tmp
    return ord_res, dist_res, cv_dist


def optspace_helper(rclr_table: np.array,
                    feature_ids: list,
                    subject_ids: list,
                    n_components: Union[int, str] = DEFAULT_COMP,
                    max_iterations: int = DEFAULT_OPTSPACE_ITERATIONS) -> (
                        OrdinationResults,
                        DistanceMatrix):
    """
    Helper function. Please use rpca directly.
    """
    # run OptSpace (RPCA)
    opt = MatrixCompletion(n_components=n_components,
                           max_iterations=max_iterations).fit(rclr_table)
    # get new n-comp when applicable
    n_components = opt.s.shape[0]
    # get PC column labels for the skbio OrdinationResults
    rename_cols = ['PC' + str(i + 1) for i in range(n_components)]
    # get completed matrix for centering
    X = opt.sample_weights @ opt.s @ opt.feature_weights.T
    # center again around zero after completion
    X = X - X.mean(axis=0)
    X = X - X.mean(axis=1).reshape(-1, 1)
    # re-factor the data
    u, s, v = svd(X)
    # only take n-components
    u = u[:, :n_components]
    v = v.T[:, :n_components]
    # calc. the new variance using projection
    p = s**2 / np.sum(s**2)
    p = p[:n_components]
    s = s[:n_components]
    # save the loadings
    feature_loading = pd.DataFrame(v, index=feature_ids,
                                   columns=rename_cols)
    sample_loading = pd.DataFrame(u, index=subject_ids,
                                  columns=rename_cols)
    # % var explained
    proportion_explained = pd.Series(p, index=rename_cols)
    # get eigenvalues
    eigvals = pd.Series(s, index=rename_cols)

    # if the n_components is two add PC3 of zeros
    # this is referenced as in issue in
    # <https://github.com/biocore/emperor/commit
    # /a93f029548c421cb0ba365b4294f7a5a6b0209ce>
    # discussed in gemelli -- PR#29
    if n_components == 2:
        feature_loading['PC3'] = [0] * len(feature_loading.index)
        sample_loading['PC3'] = [0] * len(sample_loading.index)
        eigvals.loc['PC3'] = 0
        proportion_explained.loc['PC3'] = 0

    # save ordination results
    short_method_name = 'rpca_biplot'
    long_method_name = '(Robust Aitchison) RPCA Biplot'
    ord_res = skbio.OrdinationResults(
        short_method_name,
        long_method_name,
        eigvals.copy(),
        samples=sample_loading.copy(),
        features=feature_loading.copy(),
        proportion_explained=proportion_explained.copy())
    # save distance matrix
    dist_res = DistanceMatrix(opt.distance, ids=sample_loading.index)

    return ord_res, dist_res


def rpca_table_processing(table: biom.Table,
                          min_sample_count: int = DEFAULT_MSC,
                          min_feature_count: int = DEFAULT_MFC,
                          min_feature_frequency: float = DEFAULT_MFF) -> (
                              biom.Table):
    """Filter and checks the table validity for RPCA.
    """
    # get shape of table
    n_features, n_samples = table.shape

    # filter sample to min seq. depth
    def sample_filter(val, id_, md):
        return sum(val) > min_sample_count

    # filter features to min total counts
    def observation_filter(val, id_, md):
        return sum(val) > min_feature_count

    # filter features by N samples presence
    def frequency_filter(val, id_, md):
        return (np.sum(val > 0) / n_samples) > (min_feature_frequency / 100)

    # filter and import table for each filter above
    if min_feature_count is not None:
        table = table.filter(observation_filter,
                             axis='observation',
                             inplace=False)
    if min_feature_frequency is not None:
        table = table.filter(frequency_filter,
                             axis='observation',
                             inplace=False)
    if min_sample_count is not None:
        table = table.filter(sample_filter,
                             axis='sample',
                             inplace=False)
    # check the table after filtering
    if len(table.ids()) != len(set(table.ids())):
        raise ValueError('Data-table contains duplicate sample IDs')
    if len(table.ids('observation')) != len(set(table.ids('observation'))):
        raise ValueError('Data-table contains duplicate feature IDs')
    # ensure empty samples / features are removed
    table = table.remove_empty()

    return table


def joint_rpca(tables: biom.Table,
               n_test_samples: int = DEFAULT_TESTS,
               sample_metadata: pd.DataFrame = DEFAULT_METACV,
               train_test_column: str = DEFAULT_COLCV,
               n_components: Union[int, str] = DEFAULT_COMP,
               rclr_transform_tables: bool = DEFAULT_TRNSFRM,
               min_sample_count: int = DEFAULT_MSC,
               min_feature_count: int = DEFAULT_MFC,
               min_feature_frequency: float = DEFAULT_MFF,
               max_iterations: int = DEFAULT_OPTSPACE_ITERATIONS) -> (
        OrdinationResults,
        DistanceMatrix,
        pd.DataFrame):
    """
    Performs joint-RPCA across data tables
    with shared samples.

    Parameters
    ----------
    tables: list of biom.Table, required
    A list of feature table in biom format containing shared
    samples over which metric should be computed.

    n_test_samples: int, optional : Default is 10
    Number of random samples to choose for test split samples.

    metadata: DataFrame, optional : Default is None
    Sample metadata file in QIIME2 formatting. The file must
    contain a train-test column with labels `train` and `test`
    and the row ids matched to the table(s).

    train_test_column: str, optional : Default is None
    Sample metadata column containing `train` and `test`
    labels to use for the cross-validation evaluation.

    n_components: int, optional : Default is 3
    The underlying rank of the data and number of
    output dimensions.

    rclr_transform_tables: bool, optional: If False Joint-RPCA
    will not use the RCLR transformation and will instead
    assume that the data has already been transformed
    or normalized. Default is True.

    max_iterations: int, optional : Default is 5
    The number of convex iterations to optimize the solution
    If iteration is not specified, then the default iteration is 5.
    Which reduces to a satisfactory error threshold.

    min_sample_count: int, optional : Default is 0
    Minimum sum cutoff of sample across all features.
    The value can be at minimum zero and must be an
    whole integer. It is suggested to be greater than
    or equal to 500.

    min_feature_count: int, optional : Default is 0
    Minimum sum cutoff of features across all samples.
    The value can be at minimum zero and must be
    an whole integer.

    min_feature_frequency: float, optional : Default is 0
    Minimum percentage of samples a feature must appear
    with a value greater than zero. This value can range
    from 0 to 100 with decimal values allowed.

    Returns
    -------
    OrdinationResults
        A joint-biplot of the (Robust Aitchison) RPCA feature loadings

    DistanceMatrix
        The Aitchison distance of the sample loadings from RPCA.

    DataFrame
        The cross-validation reconstruction error.

    Raises
    ------
    ValueError
        `ValueError: n_components must be at least 2`.

    ValueError
        `ValueError: max_iterations must be at least 1`.

    ValueError
        `ValueError: Data-table contains either np.inf or -np.inf`.

    ValueError
        `ValueError: The n_components must be less
            than the minimum shape of the input table`.

    """

    # filter each table
    for n, table_n in enumerate(tables):
        if rclr_transform_tables:
            tables[n] = rpca_table_processing(table_n,
                                              min_sample_count,
                                              min_feature_count,
                                              min_feature_frequency)
    # get set of shared samples
    shared_all_samples = set.intersection(*[set(table_n.ids())
                                            for table_n in tables])
    # check sample overlaps
    if len(shared_all_samples) == 0:
        raise ValueError('No samples overlap between all tables. '
                         'If using pre-transformed or normalized '
                         'tables, make sure the rclr_transform_tables '
                         'is set to False or the flag is enabled.')
    unshared_samples = set([s_n
                            for table_n in tables
                            for s_n in table_n.ids()]) - shared_all_samples
    if len(unshared_samples) != 0:
        warnings.warn('Removing %i sample(s) that do not overlap in tables.'
                      % (len(unshared_samples)), RuntimeWarning)
    # filter each table again to subset samples.
    for n, table_n in enumerate(tables):
        if rclr_transform_tables:
            table_n = table_n.filter(shared_all_samples)
            tables[n] = rpca_table_processing(table_n,
                                              min_sample_count,
                                              min_feature_count,
                                              min_feature_frequency)
        else:
            tables[n] = table_n.filter(shared_all_samples)
    shared_all_samples = set.intersection(*[set(table_n.ids())
                                            for table_n in tables])
    # rclr each table
    rclr_tables = []
    for table_n in tables:
        # perform RCLR
        if rclr_transform_tables:
            rclr_tmp = matrix_rclr(table_n.matrix_data.toarray().T).T
        # otherwise just mask zeros
        else:
            rclr_tmp = mask_value_only(table_n.matrix_data.toarray().T).T
        rclr_tables.append(pd.DataFrame(rclr_tmp,
                                        table_n.ids('observation'),
                                        table_n.ids()))
    # get training and test sample IDs
    if sample_metadata is not None and not isinstance(sample_metadata,
                                                      pd.DataFrame):
        sample_metadata = sample_metadata.to_dataframe()
    if sample_metadata is None or train_test_column is None:
        test_samples = sorted(list(shared_all_samples))[:n_test_samples]
        train_samples = list(set(shared_all_samples) - set(test_samples))
    else:
        sample_metadata = sample_metadata.loc[shared_all_samples, :]
        train_samples = sample_metadata[train_test_column] == 'train'
        test_samples = sample_metadata[train_test_column] == 'test'
        train_samples = sample_metadata[train_samples].index
        test_samples = sample_metadata[test_samples].index
    ord_res, U_dist_res, cv_dist = joint_optspace_helper(rclr_tables,
                                                         n_components,
                                                         max_iterations,
                                                         test_samples,
                                                         train_samples)
    return ord_res, U_dist_res, cv_dist


def joint_optspace_helper(tables,
                          n_components,
                          max_iterations,
                          test_samples,
                          train_samples):
    """
    Helper function for joint-RPCA
    """

    # split the tables by training and test samples
    tables_split = [[table_i.loc[:, test_samples].T,
                     table_i.loc[:, train_samples].T]
                    for table_i in tables]
    # run OptSpace
    opt_model = OptSpace(n_components=n_components,
                         max_iterations=max_iterations,
                         tol=None)
    U, s, Vs, dists = opt_model.joint_solve([[t_s.values for t_s in t]
                                             for t in tables_split])
    rename_cols = ['PC' + str(i + 1) for i in range(n_components)]
    vjoint = pd.concat([pd.DataFrame(Vs_n,
                                     index=t_n.index,
                                     columns=rename_cols)
                        for t_n, Vs_n in zip(tables, Vs)])
    ujoint = pd.DataFrame(U,
                          index=list(train_samples),
                          columns=rename_cols)
    # center again around zero after completion
    X = ujoint.values @ s @ vjoint.values.T
    X = X - X.mean(axis=0)
    X = X - X.mean(axis=1).reshape(-1, 1)
    u, s_new, v = svd(X, full_matrices=False)
    s_eig = s_new[:n_components]
    rename_cols = ['PC' + str(i + 1) for i in range(n_components)]
    v = v.T[:, :n_components]
    u = u[:, :n_components]
    # create ordination
    vjoint = pd.DataFrame(v,
                          index=vjoint.index,
                          columns=vjoint.columns)
    ujoint = pd.DataFrame(u,
                          index=list(train_samples),
                          columns=ujoint.columns)
    p = s_eig**2 / np.sum(s_eig**2)
    eigvals = pd.Series(s_eig, index=rename_cols)
    proportion_explained = pd.Series(p, index=rename_cols)
    ord_res = OrdinationResults(
            'rpca',
            'rpca',
            eigvals.copy(),
            samples=ujoint.copy(),
            features=vjoint.copy(),
            proportion_explained=proportion_explained.copy())
    # project test data into training data
    if len(test_samples) > 0:
        ord_res = transform(ord_res,
                            [t[0] for t in tables_split],
                            rclr_transform=False)
    # save results
    Udist = distance.cdist(ord_res.samples.copy(),
                           ord_res.samples.copy())
    U_dist_res = DistanceMatrix(Udist, ids=ord_res.samples.index)
    cv_dist = pd.DataFrame(dists, ['mean_CV', 'std_CV']).T
    cv_dist['run'] = 'tables_%i.n_components_%i.max_iterations_%i.n_test_%i' \
                     % (len(tables), n_components,
                        max_iterations, len(test_samples))
    cv_dist['iteration'] = list(cv_dist.index.astype(int))
    cv_dist.index.name = 'sampleid'

    return ord_res, U_dist_res, cv_dist


def transform(ordination: OrdinationResults,
              tables: biom.Table,
              subset_tables: bool = DEFAULT_MATCH,
              rclr_transform: bool = DEFAULT_TRNSFRM) -> (
        OrdinationResults):
    """
    Function to apply dimensionality reduction to table(s).
    The table(s) is projected on the first principal components
    previously extracted from a training set.

    Parameters
    ----------
    ordination: OrdinationResults
        A joint-biplot of the (Robust Aitchison) RPCA feature loadings
        produced from the training data.

    tables: list of biom.Table, required
        A list of at least one feature table in biom format containing
        shared samples over which metric should be computed.

    subset_tables: bool, optional : default is True
        Subsets the input tables to contain only features used in the
        training data. If set to False and the tables are not perfectly
        matched a ValueError will be produced.

    rclr_transform: bool, optional : default is True
        If set to false the function will expect `tables` to be dataframes
        already rclr transformed. This is used for internal functionality
        in the joint-rpca function.

    Returns
    -------
    OrdinationResults
        A joint-biplot of the (Robust Aitchison) RPCA feature loadings
        with both the input training data and new test data.

    Raises
    ------
    ValueError
        `ValueError: The input tables do not contain all
        the features in the ordination.`.

    ValueError
        `ValueError: Removing # features(s) in table(s)
        but not the ordination.`.

    ValueError
        `ValueError: Features in the input table(s) not in
        the features in the ordination.  Either set subset_tables to
        True or match the tables to the ordination.`.
    """

    # extract current U & V matrix
    Udf = ordination.samples.copy()
    Vdf = ordination.features.copy()
    s_eig = ordination.eigvals.copy().values
    # rclr each table [if needed]
    rclr_table_df = []
    if rclr_transform:
        for table_n in tables:
            rclr_tmp = matrix_rclr(table_n.matrix_data.toarray().T)
            rclr_table_df.append(pd.DataFrame(rclr_tmp,
                                              table_n.ids(),
                                              table_n.ids('observation')))
    else:
        for table_n in tables:
            rclr_table_df.append(table_n)
    rclr_table_df = pd.concat(rclr_table_df, axis=1).T
    # ensure feature IDs match
    shared_features = set(rclr_table_df.index) & set(Vdf.index)
    if len(shared_features) < len(set(Vdf.index)):
        raise ValueError('The input tables do not contain all'
                         ' the features in the ordination.')
    elif subset_tables:
        unshared_N = len(set(rclr_table_df.index)) - len(shared_features)
        warnings.warn('Removing %i features(s) in table(s)'
                      ' but not the ordination.'
                      % (unshared_N), RuntimeWarning)
    else:
        raise ValueError('Features in the input table(s) not in'
                         ' the features in the ordination.'
                         ' Either set subset_tables to True or'
                         ' match the tables to the ordination.')
    ordination.samples = transform_helper(Udf,
                                          Vdf,
                                          s_eig,
                                          rclr_table_df)
    return ordination


def transform_helper(Udf, Vdf, s_eig, table_rclr_project):
    # project new data into ordination
    table_rclr_project = table_rclr_project.reindex(Vdf.index)
    M_project = np.ma.array(table_rclr_project,
                            mask=np.isnan(table_rclr_project)).T
    M_project = M_project - M_project.mean(axis=1).reshape(-1, 1)
    M_project = M_project - M_project.mean(axis=0)
    U_projected = np.ma.dot(M_project, Vdf.values).data
    U_projected /= np.linalg.norm(s_eig)
    U_projected = pd.DataFrame(U_projected,
                               table_rclr_project.columns,
                               Udf.columns)
    return pd.concat([Udf, U_projected])


def rpca_transform(ordination: OrdinationResults,
                   table: biom.Table,
                   subset_tables: bool = DEFAULT_MATCH,
                   rclr_transform: bool = DEFAULT_TRNSFRM) -> (
        OrdinationResults):
    """
    To avoid confusion this helper function takes one input
    to use in QIIME2.
    """
    ordination = transform(ordination, [table],
                           subset_tables=subset_tables,
                           rclr_transform=rclr_transform)
    return ordination


def feature_covariance_table(ordination, features_use=None):
    """
    Function to produce a feature by feature
    covariance table from RPCA ordination
    results.

    Parameters
    ----------
    ordination: OrdinationResults
        A joint-biplot of the (Robust Aitchison) RPCA feature loadings
    features_use: list, optional : default is None
        A subset of features to use in the covariance generation.

    Returns
    -------
    DataFrame
        A feature by feature covariance table.

    """
    if features_use is not None:
        vjoint = ordination.features.copy()
        if len(set(features_use) - set(vjoint.index)) != 0:
            raise ValueError('Feature subset given contains labels'
                             ' not in the loadings.')
        vjoint = vjoint.loc[features_use, :]
    else:
        vjoint = ordination.features
    s = ordination.eigvals.values
    Vs_joint = vjoint.values @ np.diag(s)**2 @ vjoint.values.T
    joint_features = pd.DataFrame(Vs_joint,
                                  vjoint.index,
                                  vjoint.index)

    return joint_features


def feature_correlation_table(ordination: OrdinationResults) -> (pd.DataFrame):
    """
    Function to produce a feature by feature
    correlation table from RPCA ordination
    results. Note that the output can be very large in
    file size because it is all omics features by all
    omics features and is fully dense. If you would like to
    get a subset, just subset the ordination with the function
    `filter_ordination` in utils first.

    Parameters
    ----------
    ordination: OrdinationResults
        A joint-biplot of the (Robust Aitchison) RPCA feature loadings.

    Returns
    -------
    DataFrame
        A feature by feature correlation table.

    """
    joint_features = feature_covariance_table(ordination)
    # this part of the function is taken from:
    # https://gist.github.com/wiso/ce2a9919ded228838703c1c7c7dad13b
    covariance = joint_features.values
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    correlation[covariance == 0] = 0
    # convert back to dataframe
    correlation = pd.DataFrame(correlation,
                               joint_features.index,
                               joint_features.columns)
    correlation.index.name = 'featureid'
    return correlation

ModuleNotFoundError: No module named 'gemelli'